In [1]:
import os
import json
import imageio
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import transforms

from lib import utils, tri_dvgo_multiscene

Using /home/hydeng/.cache/torch_extensions as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/hydeng/.cache/torch_extensions/adam_upd_cuda/build.ninja...
Building extension module adam_upd_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module adam_upd_cuda...
Using /home/hydeng/.cache/torch_extensions as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/hydeng/.cache/torch_extensions/render_utils_cuda/build.ninja...
Building extension module render_utils_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module render_utils_cuda...


In [2]:
model_class = tri_dvgo_multiscene.DirectVoxGO
# ckpt_path = 'logs/tri_dvgo_multiscene/nerf_synthetic/rnd_liif_posemb_conv_down1/fine_last.tar'
# ckpt_path = 'logs/tri_dvgo_multiscene/nerf_synthetic/3conv_liif_pretrain_down4/fine_last.tar'
# ckpt_path = 'logs/tri_dvgo_multiscene/nerf_synthetic/3conv_liif_pretrain_down4_cosine/fine_last.tar'
ckpt_path = 'logs/tri_dvgo_multiscene/nerf_synthetic/3conv_liif_pretrain_down4_cosine_featunfold/fine_330000.tar'

# ckpt_path = 'logs/tri_dvgo_multiscene/nerf_synthetic/3MLP_liif_pretrain_down4/fine_last.tar'
# ckpt_path = 'logs/tri_dvgo_multiscene/nerf_synthetic/MLP_liif_pretrain_down4/fine_last.tar'

In [3]:
model = utils.load_model(model_class, ckpt_path)

initialized encoder networks
{'xy': Conv_Mapping(
  (body): Sequential(
    (0): ResBlock(
      (body): Sequential(
        (0): Conv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (1): ResBlock(
      (body): Sequential(
        (0): Conv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (2): ResBlock(
      (body): Sequential(
        (0): Conv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (3): ResBlock(
      (body): Sequential(
        (0): Conv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(80, 8

FileNotFoundError: [Errno 2] No such file or directory: '/home/hydeng/Documents/SR_NeRF/code/DirectVoxGO/pretrained/edsr-baseline-liif.pth'

In [ ]:
def read_image(path):
    image = imageio.imread(path)
    image = (np.array(image) / 255.).astype(np.float32)
    if image.shape[-1] == 4:
        image = image[...,:3]*image[...,-1:] + (1.-image[...,-1:])
    return image

In [ ]:
def read_poses(path, idxs):
    with open(path, 'r') as f:
        meta = json.load(f)
    poses = []
    for id in idxs:
        frame = meta['frames'][id]
        poses.append(np.array(frame['transform_matrix']))
    poses = np.stack(poses)
    return poses

In [ ]:
imagedir = 'data/nerf_synthetic/mic/train'
json_path = 'data/nerf_synthetic/mic/transforms_train.json'
idxs = [34, 49, 63]

images = []
for id in idxs:
    path = os.path.join(imagedir, f'r_{id}.png')
    images.append(read_image(path))
images = np.stack(images)

poses = read_poses(json_path, idxs)

images = torch.FloatTensor(images)
poses = torch.FloatTensor(poses)

In [ ]:
print(images.shape, poses.shape)

In [ ]:
images = images.permute(0, 3, 1, 2)
h, w = images.shape[-2:]
h, w = h // 4, w // 4
resize = transforms.Resize([h, w])
images = resize(images)
images = (images - 0.5) / 0.5

In [ ]:
feats, _, _, = model.encode_feat(images, poses)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
splits = ['xy', 'yz', 'zx']
channels = [15, 20]
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12,8))
plt.subplots_adjust(left=None,bottom=None,right=None,top=None,wspace=0.3, hspace=0)
vmin, vmax = 1e6, -1e6
for i, s in enumerate(splits):
    feats[s] = (feats[s] - feats[s].min()) / (feats[s].max() - feats[s].min())
    
    vmin = min(feats[s].min().item(), vmin)
    vmax = max(feats[s].max().item(), vmax)

for i, channel in enumerate(channels):
    for j, s in enumerate(splits):
        axes[i, j].imshow(feats[s][0][channel].detach().numpy(), cmap='RdBu')
        axes[i, j].xaxis.set_tick_params(labelsize=8)
        axes[i, j].yaxis.set_tick_params(labelsize=8)

rows = [f'channel {i}' for i in channels]
cols = splits
for ax, row in zip(axes[:,0], rows):
    ax.set_ylabel(row, rotation=90, fontsize=10)

for ax, col in zip(axes[0], cols):
    ax.set_title(col, fontsize=10)